In [1]:
import os
import re
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
def openDoc(file):
    ''' Open the HTML document
    @param file: file_path, str
    @return soup: beautiful soup, bs4.BeautifulSoup
    '''
    htmldoc = open(file, 'r', encoding='utf-8')
    htmlhandle = htmldoc.read()
    soup = BeautifulSoup(htmlhandle, 'lxml')
    htmldoc.close()
    return soup

In [3]:
def splitContent(case_content):
    ''' Split Use Case Contents
    @param case_content: use case contents, str
    @return desc_list: use case description, list
    '''
    text = re.sub(r'[\n\xa0]', '', case_content)
    desc_list = re.split(r'\x95\s+', text)
    return desc_list

In [4]:
def getUseCase(page_code, use_case):
    ''' Analysis Use Case Contents
    @param page_code: page code, str
    @param use_case: use case, bs4.element.Tag
    @return case_desc: use case desc, list
    @return child_case: child use case, list
    '''
    case_content = use_case.get_text()
    tmp = re.split(r'#', case_content)
    if len(tmp) > 1:
        case_note = re.sub(r'[\n\xa0]', '', tmp[1])
    else:
        case_note = ''
    desc_list = splitContent(tmp[0])
    
    case_desc = [page_code,]
    case_desc.extend(re.split(r'\s+', desc_list[0]))
    case_desc.append(case_note)
    case_code = case_desc[1]
    
    child_case = []
    for desc in desc_list[1:]:
        tmp = re.split(r':\s+', desc)
        num = tmp[0]
        name = tmp[1]
        child = [page_code, case_code, num, name]
        child_case.append(child)
    
    return case_desc, child_case

In [5]:
def getPageCase(soup):
    ''' Analysis all use cases for the page
    @param soup: beautiful soup, bs4.BeautifulSoup
    '''
    base = soup.find('div', id='base')
    overview = base.find_all('div', attrs={'data-label': '页面说明'})
    normal = base.find_all('div', attrs={'data-label': '普通用例'})
    abnormal = base.find_all('div', attrs={'data-label': '异常用例'})
    logic = base.find_all('div', attrs={'data-label': '数据逻辑'})
    
    if overview:
        overview_desc = splitContent(overview[0].get_text())
        page = re.split(r'\s+', overview_desc[0])
        page.extend(overview_desc[1:])
        page_code = page[0]
        page_list.append(page)
    
    if normal:
        for case in normal:
            case_desc, child_case = getUseCase(page_code, case)
            case_desc.append('normal')
            case_list.append(case_desc)
            child_case_list.extend(child_case)
        
    if abnormal:
        for case in abnormal:
            case_desc, child_case = getUseCase(page_code, case)
            case_desc.append('abnormal')
            case_list.append(case_desc)
            child_case_list.extend(child_case)
        
    if logic:
        for case in logic:
            case_desc, child_case = getUseCase(page_code, case)
            case_desc.append('logic')
            case_list.append(case_desc)
            child_case_list.extend(child_case)

In [6]:
file_list = []
ignore_list = ['index.html', 'page1.html', 'page2.html', 'page3.html', 'start.html', 'start_c_1.html', 'start_g_0.html']

file_dir = 'F:/profile/iSpider/practica/03_Axure/demo/'
file_type = '.html'
for file in os.listdir(file_dir):
    if file_type in file:
        if file not in ignore_list:
            file = file_dir + file
            file_list.append(file)

In [7]:
page_list = []
case_list = []
child_case_list = []

In [8]:
for file in file_list:
    getPageCase(openDoc(file))

In [9]:
page_list

[['A-0', '页面名称', '0', '用户场景：用户进入该页面的场景', '功能描述：页面的功能', '补充说明：需求的补充说明']]

In [10]:
case_list

[['A-0', 'N1', '普通用例名称', '0', '', 'normal'],
 ['A-0', 'N2', '普通用例名称', '0', ' 这里是备注', 'normal'],
 ['A-0', 'E1', '异常用例名称', '0', '', 'abnormal'],
 ['A-0', 'D1', '数据逻辑', '0', '', 'logic']]

In [11]:
child_case_list

[['A-0', 'N1', 'N1-1', '普通子用例说明'],
 ['A-0', 'N1', 'N1-2', '普通子用例说明，包括：- 这里是具体的说明- 这里是具体的说明'],
 ['A-0', 'N1', 'N1-3', '普通子用例说明'],
 ['A-0', 'N2', 'N2-1', '普通子用例说明'],
 ['A-0', 'N2', 'N2-2', '普通子用例说明，包括：- 这里是具体的说明- 这里是具体的说明'],
 ['A-0', 'N2', 'N2-3', '普通子用例说明，普通子用例说明，普通子用例说明'],
 ['A-0', 'E1', 'E1-1', '异常子用例说明'],
 ['A-0', 'E1', 'E1-2', '异常子用例说明，包括：- 这里是具体的说明- 这里是具体的说明'],
 ['A-0', 'D1', 'D1-1', '排序规则'],
 ['A-0', 'D1', 'D1-2', '状态变更规则'],
 ['A-0', 'D1', 'D1-3', '字段说明，包括：- 这里是具体说明- 这里是具体说明']]

In [12]:
page_df = pd.DataFrame(page_list, columns=['page_code', 'page_name', 'page_release', 'scene', 'page_desc', 'page_note'])
case_df = pd.DataFrame(case_list, columns=['page_code', 'case_code', 'case_name', 'case_release', 'case_note', 'case_type'])
child_case_df = pd.DataFrame(child_case_list, columns=['page_code', 'case_code', 'child_case_code', 'child_case_desc'])

In [13]:
# 查看表信息
def showTableInfo(db, table):
    ''' Show columns from table
    @param db: database name, str
    @param table: table name, str
    @return df: table structure, DataFrame
    '''
    conn = sqlite3.connect(db)
    df = pd.read_sql('PRAGMA table_info(%s)' % table, con=conn)
    return df

In [14]:
# 对数据库进行提交操作
def commit_sql(db, sql):
    ''' Save changes to the database
    @param db: database name, str
    @param sql: sql statement, str
    '''
    conn = sqlite3.connect(db)
    
    cursor = conn.cursor() #创建 Cursor
    cursor.execute(sql)
    cursor.close()
    
    conn.commit()
    conn.close()
    
    print("提交成功")

In [15]:
import sqlite3
db = 'axure.db'
conn = sqlite3.connect(db)

In [16]:
# 创建  page 表
create_page = '''
    CREATE TABLE page (
        page_id INTEGER PRIMARY KEY AUTOINCREMENT, --自增主键
        page_code VARCHAR(16), 
        page_name VARCHAR(32),
        page_release VARCHAR(8),
        scene TEXT,
        page_desc TEXT, 
        page_note TEXT
    )'''
commit_sql(db, create_page)

# 创建  use_case 表
create_case = '''
    CREATE TABLE use_case (
        case_id INTEGER PRIMARY KEY AUTOINCREMENT, --自增主键
        page_code VARCHAR(16), 
        case_code VARCHAR(16),
        case_name VARCHAR(32),
        case_release VARCHAR(8),
        case_note TEXT,
        case_type VARCHAR(8)
    )'''
commit_sql(db, create_case)

# 创建 child_case 表
create_child_case = '''
    CREATE TABLE child_case (
        child_case_id INTEGER PRIMARY KEY AUTOINCREMENT, --自增主键
        page_code VARCHAR(16), 
        case_code VARCHAR(16),
        child_case_code VARCHAR(16),
        child_case_desc TEXT
    )'''
commit_sql(db, create_child_case)

提交成功
提交成功
提交成功


In [17]:
showTableInfo(db, 'page')

,cid,name,type,notnull,dflt_value,pk
0,0,page_id,INTEGER,0,None,1
1,1,page_code,VARCHAR(16),0,None,0
2,2,page_name,VARCHAR(32),0,None,0
3,3,page_release,VARCHAR(8),0,None,0
4,4,scene,TEXT,0,None,0
5,5,page_desc,TEXT,0,None,0
6,6,page_note,TEXT,0,None,0


In [18]:
page_df.to_sql(name='page', con=conn, if_exists='append', index=False)
case_df.to_sql(name='use_case', con=conn, if_exists='append', index=False)
child_case_df.to_sql(name='child_case', con=conn, if_exists='append', index=False)